## Import libs

In [1]:
import networkx as nx # the main libary we will use
from networkx.algorithms import bipartite
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit 
from itertools import combinations
from nxviz.plots import CircosPlot
from nxviz import ArcPlot
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce
import dask
import dask.array as da, dask.dataframe as dd

## Create Bipartite Graph

In [15]:
def create_graph(users, movies, df):
    B = nx.Graph()

    # Add nodes with the node attribute "bipartite"
    B.add_nodes_from(users, bipartite=0)
    B.add_nodes_from(movies, bipartite=1)

    # Add edges only between nodes of opposite node sets
    B.add_weighted_edges_from(df["edge"].values.tolist(), weight="rating")

    #check that the node set is actually correct and that the input graph is actually bipartite.
    print(nx.is_connected(B))
    print(nx.is_bipartite(B))
    print ('Number of nodes: {}'.format(B.number_of_nodes()))
    print ('Number of edges: {}'.format(B.number_of_edges()))
    return B

## Project one side of the graph:

In [16]:
def project_side(B,type_v, def_group):
    G_movies = bipartite.weighted_projected_graph(B, type_v, ratio=True)
    attributes_dictionary = def_group.to_dict('index')
    nx.set_node_attributes(G_movies, attributes_dictionary)
    return B

## Plot the graph

In [20]:
def plot_g(G):
    nx.draw(G, pos=nx.spring_layout(G_movies), with_labels = True, node_color = "#00CCFF")
    plt.show()

def CircosPlot(G, grouping):
    c = CircosPlot(
        G,
        dpi=600,
        node_grouping=grouping,
        edge_width="count",
        figsize=(20, 20),
        node_color=grouping,
        node_labels=True,
    )
    c.draw()
    plt.show()

def ArcPlot(G, grouping):
    a = ArcPlot(G, node_color=grouping, node_grouping=grouping)
    a.draw()

## Graph attributes

In [22]:
def plot_degree_dist(G):
    degree_hist = nx.degree_histogram(G) 
    print (degree_hist)
    degree_hist = np.array(degree_hist, dtype=float)
    degree_prob = degree_hist/G.number_of_nodes()
    # plotting
    fig = plt.figure(figsize=(6,6))
    axes = fig.add_axes([1,1,1,1])
    
    axes.loglog(np.arange(degree_prob.shape[0]), degree_prob, 'b.', markersize=15, alpha=0.5)
    
    axes.set_xlabel('k')
    axes.set_ylabel('p(k)')
    axes.set_title('Degree Distribution')
    
    plt.show()

### Plot knn as function of k

Fit function of the form:  $a\cdot X^{\mu}$

In [23]:
def fit_func(x,a,mu):
    return (a*x)**mu

In [24]:
def plot_knn(G, fit=True): 
    knn_dict = nx.k_nearest_neighbors(G) # k_nearest_neighbors return dict with knn for each k
    k_lst = sorted(knn_dict.keys())
    knn_lst = []
    for k in k_lst:
        knn_lst.append(knn_dict[k])
    
    # plotting
    fig = plt.figure(figsize=(6,6))
    axes = fig.add_axes([1,1,1,1])
    
    axes.loglog(k_lst,knn_lst,'b.', markersize=15, alpha=0.5)
    
    axes.set_xlabel('k')
    axes.set_ylabel('knn(k)')
    axes.set_title('Average next neighbor degree')
    try:
        if fit:
            # fit a*x^mu
            popt, pcov = curve_fit(fit_func, np.array(k_lst), np.array(knn_lst))
            axes.loglog(np.array(k_lst), fit_func(np.array(k_lst), *popt), '--', c='gray')

        plt.show()
    except:
        print ("RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 600.")
        

### Plot snn as function of k

In [25]:
def plot_snn(G, fit=True): 
    snn_dict = nx.k_nearest_neighbors(G, weight='weight') 
    k_lst = sorted(snn_dict.keys())
    snn_lst = []
    for k in k_lst:
        snn_lst.append(snn_dict[k])
    
    # plotting
    fig = plt.figure(figsize=(6,6))
    axes = fig.add_axes([1,1,1,1])
    
    axes.loglog(k_lst,snn_lst,'b.', markersize=15, alpha=0.5)
    
    axes.set_xlabel('k')
    axes.set_ylabel('snn(k)')
    axes.set_title('Average next neighbor strength')
    try:
        if fit:
            # fit a*x^mu
            popt, pcov = curve_fit(fit_func, np.array(k_lst), np.array(snn_lst))
            axes.loglog(np.array(k_lst), fit_func(np.array(k_lst), *popt), '--', c='gray')

        plt.show()
    except:
        print ("RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 600.")

### Clustering coefficient for k
calculate C(k) the average clustering coefficient for nodes with degree k.

In [26]:
def plot_clustering_coefficient(G):
    clustering_dict = {}
    for node in G.nodes():
        k = G.degree(node)
        if not k in clustering_dict:
            clustering_dict[k] = [nx.clustering(G,node)]
        else:
            clustering_dict[k].append(nx.clustering(G,node))
    k_lst = sorted(clustering_dict.keys())
    clustering_lst = []
    for k in k_lst:
        clustering_lst.append(np.array(clustering_dict[k]).mean())
    
    # plotting
    fig = plt.figure(figsize=(6,6))
    axes = fig.add_axes([1,1,1,1])
    
    axes.loglog(k_lst,clustering_lst,'b.', markersize=15, alpha=0.5)
    
    axes.set_xlabel('k')
    axes.set_ylabel('C(k)')
    axes.set_title('Average clustering coefficient')
    
    plt.show()

In [ ]:
def print_attributes(G):
# Diameter, Connected, Degree Assortativity Coefficient
    dict_data = {}
    info = nx.info(G)
    print (info)
    splited = info.splitlines()
    for s in splited:
        split = s.strip().split(":")
        dict_data[split[0]]=split[1]       
    dac = nx.degree_assortativity_coefficient(G)
    print ('Degree Assortativity Coefficient (r): %s' % dac)
    if not nx.is_directed(G):
        if nx.is_connected(G):
            diameter = nx.diameter(G)
            print ('Diameter: %s' % diameter) # print diameter of the network
            dict_data["Diameter"] = diameter
            return dict_data
        else:
            print ('Graph not connected: infinite path length')
            large_comp = len(max(nx.connected_components(G), key=len))
            print ('Size of largest component: %s' % large_comp)
            dict_data["largest component"] = large_comp
            return dict_data
    

def plot_hist(list_to_plot,x_label):
    plt.hist(list_to_plot)
    plt.ylabel('Count')
    plt.xlabel(x_label)
    plt.show()
        
def calculate_degree(G):
    d = dict(G.degree(G.nodes()))
    nx.set_node_attributes(G, d, 'degree')
    df_degree = pd.DataFrame.from_dict(d, orient='index', columns=['degree']).sort_values(by='degree', ascending=False)
    
    #plot_degree
    #plot_hist(d.values(),'degree')
    
    return G, df_degree
    
def calculate_degree_centrality(G):
    print ("\nCalculating Degree Centrality...")
    dc = nx.degree_centrality(G)
    nx.set_node_attributes(G, dc, 'degree_cent')
    df_degree_centrality = pd.DataFrame.from_dict(dc, orient='index', columns=['degree_centrality']).sort_values(by='degree_centrality', ascending=False)
    
    #plot_degree_centrality
    #plot_hist(dc.values(),'degree_centrality')
    
    return G, df_degree_centrality

## Finding Major Movies (PageRank)
#identify the key or most “important” entities in the graph. 
#To do this, we will run the PageRank algorithm on top of the graph. This will assign a score to each node based on the structure of the incoming edges.

def page_rank(G):    
    # run PageRank on the graph. algorithm returns a dictionary with the scores
    page_rank_dict = nx.pagerank(G)
    nx.set_node_attributes(G, page_rank_dict, 'page rank')
    df_page_rank = pd.DataFrame.from_dict(page_rank_dict, orient='index', columns=['Page Rank']).sort_values(by='Page Rank', ascending=False)
    
    #plot_page_rank
    #plot_hist(page_rank_dict.values(),'page rank')
    return G, df_page_rank
    #print 'Max Pagerank player is {}:{}, Score:{}'.format(max_node_p,G.node[max_node_p]['title'],max_score)
    
def clustering_coefficient(G):
    clustering = nx.clustering(G)
    nx.set_node_attributes(G, clustering, 'clustering coefficient')
    df_clustering_coefficient = pd.DataFrame.from_dict(clustering, orient='index', columns=['clustering coefficient']) # get clustering coefficient of all nodes
    print ("average clustering coefficient"+ str(nx.average_clustering(G))) # get average clustering coefficient
    
    #plot_clustering_coefficient
    #plot_hist(clustering.values(),'clustering coefficient')
    return G, df_clustering_coefficient

def calculate_eigenvector_centrality(graph):
    print ("\n\tCalculating Eigenvector Centrality...")
    g = graph
    #ec = nx.eigenvector_centrality_numpy(g)
    #nx.set_node_attributes(g, 'eigenvector', ec)
    eigenvector_dict = nx.eigenvector_centrality(G) # Run eigenvector centrality
    nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')
    df_eigenvector = pd.DataFrame.from_dict(eigenvector_dict, orient='index', columns=['eigenvector']).sort_values(by='eigenvector', ascending=False)
    
    #plot_eigenvector
    #plot_hist(eigenvector_dict.values(),'eigenvector')
    plt.show()
    
    return G, df_eigenvector

## Betweenness Centrality
#finding the key intermediary nodes in terms of how information may flow inside the graph.
#Vertices with high betweenness centrality, means that they have a large influence in the connectivity of their neighbors with the other nodes in the graph.

def betweennes(G):    
    betweenness_dict = nx.betweenness_centrality(G)
    nx.set_node_attributes(G, betweenness_dict, 'betweenness')
    df_betweeness_rank = pd.DataFrame.from_dict(betweenness_dict, orient='index', columns=['betweennes']).sort_values(by='betweennes', ascending=False)
    
    #plot_betweennes
    #plot_hist(betweenness_dict.values(),'betweenness')
    
    return G, df_betweeness_rank
    #print 'Max betweenness_centrality player is {}:{}, Score:{}'.format(max_node_b,G.node[max_node_b]['name'],max_score)

In [28]:
def plot_parameters(top_df, sort_column, column_by ,title):
    
    #fig, ax = plt.subplots(figsize=(10, 8), dpi=600)
    
    #sns.barplot(x=column_by, y="value", data=top_df, hue="parameter", palette='winter_r', orient='v')
    sns.catplot(x="value", y=column_by, data=top_df, hue="parameter", palette='winter_r', kind='bar', orient='h')
    # "Top 10 Authors, normalized parameters"
    plt.tight_layout()
    plt.title(title)
    plt.legend(bbox_to_anchor=(0, 1), loc='upper left', ncol=1)
    plt.show()

###  Nodes with highest parameter value

In [29]:
# find nodes with highest score
def find_nodes_with_highest_score(df, score):
    max_score = df[score].max()
    df_max_nodes = df[df[score]==max_score]
    return df_max_nodes

## Cluester similar movies

In [30]:
def community_best_partition(G):
    communities = community.best_partition(G)
    nx.set_node_attributes(G, communities, 'modularity')
    return G

def community_kernighan_lin_bisection(G):
    KL_communities_generator = community.kernighan_lin_bisection(G,max_iter=10)
    for s in KL_communities_generator:
        print (s)
        
def community_girvan_newman(G):  
    GN_communities_generator = community.girvan_newman(G)
    top_level_communities = next(GN_communities_generator)
    next_level_communities = next(GN_communities_generator)
    GN_comm_sets = sorted(map(sorted, next_level_communities))

def modularity(G,communites):
    return community.modularity(G,communites)

## Friend Recommendation: Open Triangles
we want to see if we can do some friend recommendations by looking for open triangles.
Open triangles are - A knows B and B knows C, but C's relationship with A isn't captured in the graph.
What are the two general scenarios for finding open triangles that a given node is involved in?
The given node is the centre node.
The given node is one of the termini nodes.

In [31]:
def get_open_triangles(G, node):
    open_triangle_nodes = []
    neighbors = set(G.neighbors(node))
    
    for n1, n2 in combinations(neighbors, 2):
        if not G.has_edge(n1, n2):
            open_triangle_nodes.append([n1, node, n2])
            frieds_pair_recomded.append((n1, n2))
    preds = nx.jaccard_coefficient(G, frieds_pair_recomded)
#     for u, v, p in preds:
#         '(%d, %d) -> %.8f' % (u, v, p)
    df = pd.DataFrame(list(preds),columns=['n1','n2','jaccard']).sort_values(by=['n1','n2','jaccard'])
    max_jaccard_by_user = df.groupby('n')['jaccard'].max().reset_index(name='max')
    return open_triangle_nodes, max_jaccard_by_user

# find shared movies: Transposition

In [32]:
def user_user_projection_matrix(G):
    #compute adjacency matrix
    # Get the list of people and list of clubs from the graph: users_nodes, movies_nodes
    users_nodes = get_nodes_from_partition(G, 'users')
    movies_nodes = get_nodes_from_partition(G, 'movies')

    # Compute the biadjacency matrix: bi_matrix
    bi_matrix = nx.bipartite.biadjacency_matrix(G, row_order=users_nodes, column_order=movies_nodes)

    # Compute the user-user projection: user_matrix
    user_matrix = bi_matrix @ bi_matrix.T

    return users_nodes, user_matrix

In [33]:
def sharing(user_matrix, user_nodes):
    # Find out the users who rated the most number of movies
    diag = user_matrix.diagonal() 
    indices = np.where(diag == diag.max())[0]  
    print('Number of movies: {0}'.format(diag.max()))
    print('Users with the most # rating:')
    for i in indices:
        print('- {0}'.format(user_nodes[i]))

    # Set the diagonal to zero and convert it to a coordinate matrix format
    user_matrix.setdiag(0)
    users_coo = user_matrix.tocoo()

    # Find pairs of users who shared rating in the most number of movies
    indices = np.where(users_coo.data == users_coo.data.max())[0]
    print('Users with most shared movies:')
    for idx in indices:
        print('- {0}, {1}'.format(user_nodes[users_coo.row[idx]], user_nodes[users_coo.col[idx]]))

## Graph differences over time

In [ ]:
def g_difference(Gs):
    # Instantiate a list of graphs that show edges added: added
    added = []
    # Instantiate a list of graphs that show edges removed: removed
    removed = []
    # Here's the fractional change over time
    fractional_changes = []
    window = 1  
    i = 0      

    for i in range(len(Gs) - window):
        g1 = Gs[i]
        g2 = Gs[i + window]

        # Compute graph difference here
        added.append(nx.difference(g2, g1))   
        removed.append(nx.difference(g1, g2))

        # Compute change in graph size over time
        fractional_changes.append((len(g2.edges()) - len(g1.edges())) / len(g1.edges()))

    # Print the fractional change
    print(fractional_changes)

# plot number of edge changes over time

In [ ]:
def plot_diffrences(added, removed, fractional_changes):
    fig = plt.figure()
    ax1 = fig.add_subplot(111)

    # Plot the number of edges added over time
    edges_added = [len(g.edges()) for g in added]
    plot1 = ax1.plot(edges_added, label='added', color='orange')

    # Plot the number of edges removed over time
    edges_removed = [len(g.edges()) for g in removed]
    plot2 = ax1.plot(edges_removed, label='removed', color='purple')

    # Set yscale to logarithmic scale
    ax1.set_yscale('log')  
    ax1.legend()

    # 2nd axes shares x-axis with 1st axes object
    ax2 = ax1.twinx()

    # Plot the fractional changes over time
    plot3 = ax2.plot(fractional_changes, label='fractional change', color='green')

    # Here, we create a single legend for both plots
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines1 + lines2, labels1 + labels2, loc=0)
    plt.axhline(0, color='green', linestyle='--')
    plt.show()

# plot number of edges over time

In [ ]:
def plot_num_edges_over_time(edge_sizes, year):    
    fig = plt.figure()
    # Plot edge sizes over time
    plt.plot(edge_sizes)
    plt.xlabel('Time elapsed from year %s.'% str(year)) 
    plt.ylabel('Number of edges')                           
    plt.show()

# degree centrality over time

In [ ]:
def dc_time(cents,start,end):
    # Plot ECDFs over time
    fig = plt.figure()
    for i in range(start,end+1):
        x, y = ECDF(cents[i].values()) 
        plt.plot(x, y, label='Year {0}'.format()) 
    plt.legend()   
    plt.show()

### Get data and add features

In [2]:
#path = r"sample_100k (1).csv"
path = r"Data/25k_users_sample.csv"
#path = r"final_100k_users_sample.csv"
df = pd.read_csv(path)
df_movies = pd.read_csv(r"Data/movie_titles.csv", names=["year","title"], index_col=0, encoding="ISO-8859-1",parse_dates=True)

In [3]:
df["movie"] = df["movie"].astype(str)
df["date"] = pd.to_datetime(df["date"])
df_movies.fillna(0,inplace=True)
df_movies["year"] = df_movies["year"].astype("int64")
df_movies["movie"] = df_movies.index
df = df.merge(df_movies, on="movie", how="left")

In [9]:
df = dd.from_pandas(df, npartitions=2*4)

In [10]:
df = dd.from_pandas(df, npartitions=2*4)
df[['week_day', 'year_rating', 'month', 'day_month']] = df.apply(dateTime_to_dat_year_month_date_time, axis=1, result_type='expand')
df = df.compute()
df["edge"] = df.apply(create_edge,axis=1)

C:\Users\or\Anaconda3\lib\site-packages\dask\dataframe\core.py:3152: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta={0: 'int64', 1: 'int64', 2: 'int64', 3: 'int64'})

  warnings.warn(meta_warning(meta))


In [11]:
df.to_csv(r"C:\Users\or\Documents\final_25k_users_sample.csv")

## Desribe data

In [13]:
df.head()

,user,rating,date,movie,year,title,week_day,year_rating,month,day_month,edge
0,2625420,2.0,2004-05-25,13368,1999,Sarfarosh,1,2004,5,25,"(2625420, 13368, 2.0)"
1,1650301,1.0,2005-08-30,13368,1999,Sarfarosh,1,2005,8,30,"(1650301, 13368, 1.0)"
2,2500511,4.0,2003-08-11,13368,1999,Sarfarosh,0,2003,8,11,"(2500511, 13368, 4.0)"
3,2473764,4.0,2005-09-26,13368,1999,Sarfarosh,0,2005,9,26,"(2473764, 13368, 4.0)"
4,310049,2.0,2004-03-15,13368,1999,Sarfarosh,0,2004,3,15,"(310049, 13368, 2.0)"


## Create a bipartite graph, nodes and edges

In [27]:
path = r"Data\final_25k_users_sample.csv"

In [28]:
data = pd.read_csv(path,index_col=0)
data["movie"] = data["movie"].astype(str)

C:\Users\or\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [29]:
def transform_str_to_tuple(val):
    tup = list(val[1:-1].split(","))
    tup[0] = int(tup[0])
    tup[1] = str(tup[1]).replace("'","").strip()
    tup[2] = float(tup[2])
    return tuple(tup)

In [30]:
data['edge'] = data['edge'].apply(transform_str_to_tuple)

In [34]:
df_list_per_year = [x for _, x in data.groupby(data['year_rating'])]

In [40]:
df_list_per_year_above2002 = df_list_per_year[4:]

In [42]:
list_B = []
list_Gmovies = []
for data in df_list_per_year_above2002:
    print (data["year_rating"].unique().tolist()[0])
    users = data["user"].unique().tolist()
    movies = data["movie"].unique().tolist()
    # coumt/mean rating for each movie and count/mean rating for each user
    df_per_movie = data.groupby(["movie","title","year"]).agg({'rating':"mean", 'user':'count'}).reset_index().rename(columns={'rating':'mean_rating','user' : '#rating'}).sort_values(by='#rating').reset_index(drop=True)
    df_per_user = data.groupby(["user"]).agg({'rating':"mean", 'movie':'count'}).reset_index().rename(columns={'rating':'mean_rating','movie' : '#rating'}).sort_values(by='#rating').reset_index(drop=True)
    
    B = create_graph(users, movies, data)
    G_users = project_side(B, movies, df_per_movie)
    list_B.append(B)
    list_Gmovies.append(G_users)

2003
True
True
Number of nodes: 11809
Number of edges: 429321
2004
True
True
Number of nodes: 20808
Number of edges: 1261715
2005
True
True
Number of nodes: 28420
Number of edges: 1764376


In [ ]:
create_graph(df.user, movies, data)

In [ ]:
d = print_attributes(list_Gmovies[0])

Name: 
Type: Graph
Number of nodes: 11809
Number of edges: 429321
Average degree:  72.7108


In [45]:
for G_movies in list_Gmovies:
    print_attributes(G_movies)
    edge_sizes    

Name: 
Type: Graph
Number of nodes: 11809
Number of edges: 429321
Average degree:  72.7108


KeyboardInterrupt: 

In [ ]:
plot_degree_dist(G_movies)
plot_snn(G_movies, fit=True)
plot_knn(G_movies, fit=True)
if not nx.is_directed(G):
    plot_clustering_coefficient(G_movies)

In [ ]:
# Create a list of degree centrality scores year-by-year
cents = []
G, df_degree = calculate_degree(G_movies)
G, df_degree_centrality = calculate_degree_centrality(G)
G, df_page_rank = page_rank(G)
G, df_clustering = clustering_coefficient(G)
G, df_eigenvector = calculate_eigenvector_centrality(G)
G, df_betweeness_rank = betweennes(G)
dfs = [df_degree, df_degree_centrality, df_page_rank, df_clustering, df_eigenvector, df_betweeness_rank]
df_final = reduce(lambda left,right: pd.merge(left, right,on='name'), dfs)

nx.write_gexf(G, "movies_25_graph.gexf")

In [ ]:
#plot scatter of degree vs. degree_centrality
plt.scatter(x=df_degree['degree'], y=df_degree_centrality['degree_centrality'])

In [ ]:
df_final = df_final.merge(df_per_movie, left_index=True, right_index=True)
df_final.to_csv("graph_calculation.csv")
df_final.drop(['year', 'movie'], axis=1, inplace=True)

In [ ]:
title = "Top 10 normalized parameters"
for col in df_final.columns:
    if col!='title':
        df_final[col] = df_final[col] / max(df_final[col])
df_final.set_index('title',inplace=True)

top_df = df_final.sort_values('mean_rating', ascending=False)[:10]
top_df = pd.DataFrame(top_df.stack()).reset_index()
top_df.columns = ['title', "parameter", "value"]
#top_df.set_index('title',inplace=True)
top_df['title'] = top_df['title'].astype("category") 
top_df['value'] = top_df['value'].apply(pd.to_numeric)
plot_parameters(top_df, 'mean_rating', 'title', title)

In [ ]:
df_final.hist(bins=15,figsize=(10,10), density=True, align='mid',log =True,grid =False)

In [ ]:
G = community_best_partition(G)
#plotting the communities
plot_g(G)
CircosPlot(G, 'communities')
ArcPlot(G, 'communities')

In [ ]:
get_open_triangles(G, node)
users_nodes, user_matrix = user_user_projection_matrix(G)  
sharing(user_matrix, user_nodes)